In [38]:
import requests
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [17]:
config_file = "config.json"
with open(config_file, "r") as input_file:
    config = json.load(input_file)

In [40]:
# read consumer data
request_consumers = f"{config['consumer_api']}/consumer/all"
result = requests.get(request_consumers)
consumers = json.loads(json.loads(result.text))
# read network data
initialize_network = f"{config['network_api']}/initialize"
request_initialize = requests.get(initialize_network)

request_network = f"{config['network_api']}/network"
result = requests.get(request_network)
network = json.loads(json.loads(result.text))

unix_time_seconds = 1692347305
request_opf = f"{config['network_api']}/opf/{unix_time_seconds}"
result = requests.get(request_network)
opf = json.loads(json.loads(result.text))

# create pandas data frame
data = {}
data['name'] = [x['name'] for x in consumers]
data['identifier'] = [x['identifier'] for x in consumers]
data['latitude'] = [x['latitude'] for x in consumers]
data['longitude'] = [x['longitude'] for x in consumers]
data['category'] = [x['category'] for x in consumers]
data['network_entity'] = [x['network_entity'] for x in consumers]

data['name'] += ([x['name'] for x in network['entities']])
data['identifier'] += ([x['identifier'] for x in network['entities']])
data['latitude'] += ([x['latitude'] for x in network['entities']])
data['longitude'] += ([x['longitude'] for x in network['entities']])
data['category'] += ([x['category'] for x in network['entities']])
data['network_entity'] += ([x['network_entity'] for x in network['entities']])

df_nodes = pd.DataFrame(data=data)

data_lines = {}
# data_lines['']

KeyError: ''

In [35]:
network['lines']

[{'from_bus': 'ec28306c-772c-4d65-9916-55c03ce61566',
  'to_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f',
  'std_type': 'NAYY 4x50 SE',
  'length_km': 0.1},
 {'from_bus': '60d36bfa-92a2-4521-95fa-ed0e95f2413f',
  'to_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2',
  'std_type': 'NAYY 4x50 SE',
  'length_km': 0.1},
 {'from_bus': 'e6c8200f-d84b-499b-ad13-4313eec39ca2',
  'to_bus': 'd3ae8923-d4cd-46fb-ae6c-e4a20c7e5dab',
  'std_type': 'NAYY 4x50 SE',
  'length_km': 0.1}]

In [37]:
fig = px.scatter_mapbox(df_nodes, lat="latitude", lon="longitude", color="category",  color_continuous_scale=px.colors.cyclical.IceFire, zoom=15, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [25]:
fig